In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
### packages for geo data 
import requests
import osmnx as ox
from pandana.loaders import osm
%matplotlib inline
import re

In [2]:
url = 'https://raw.githubusercontent.com/amac-lfc/airbnb/master/listings_cleaned.csv?token=AHV2IBERPQTIJA36632SDGC7A6YOW'
data = pd.read_csv(url,index_col=['id'])

In [3]:
ox.config(log_console=True, use_cache=True)

In [4]:
tags = {'amenity':['restaurant', 'bar', 'cafe']}
test = ox.pois_from_point(point=(data.latitude.iloc[0],data.longitude.iloc[0]), dist=500, tags=tags)
test.head()

C:\Users\sgevo\Anaconda3\envs\ox\lib\site-packages\osmnx\pois.py:370: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf = gdf.loc[gdf["geometry"].centroid.within(polygon)]


,osmid,geometry,amenity,brand,brand:wikidata,brand:wikipedia,cuisine,name,official_name,takeaway,...,description,operator,addr:unit,nodes,addr:street:name,addr:street:prefix,addr:street:type,building,chicago:building_id,building:levels
2465890859,2465890859,POINT (-87.63018 41.90379),cafe,Starbucks,Q37158,en:Starbucks,coffee_shop,Starbucks,Starbucks Coffee,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2480059861,2480059861,POINT (-87.63905 41.90489),cafe,Starbucks,Q37158,en:Starbucks,coffee_shop,Starbucks,Starbucks Coffee,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2480104438,2480104438,POINT (-87.63465 41.90404),restaurant,NaN,NaN,NaN,english,Glunz Tavern,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3076177733,3076177733,POINT (-87.62850 41.90178),restaurant,NaN,NaN,NaN,NaN,Dublins Bar and Grill,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3076179333,3076179333,POINT (-87.62809 41.89932),cafe,Le Pain Quotidien,Q2046903,en:Le Pain Quotidien,coffee_shop,Le Pain Quotidien,NaN,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
test.columns

Index(['osmid', 'geometry', 'amenity', 'brand', 'brand:wikidata',
       'brand:wikipedia', 'cuisine', 'name', 'official_name', 'takeaway',
       'element_type', 'addr:city', 'addr:housenumber', 'addr:postcode',
       'addr:state', 'addr:street', 'opening_hours', 'phone', 'website',
       'addr:country', 'wheelchair', 'delivery', 'description', 'operator',
       'addr:unit', 'nodes', 'addr:street:name', 'addr:street:prefix',
       'addr:street:type', 'building', 'chicago:building_id',
       'building:levels'],
      dtype='object')

In [6]:
test.amenity.dropna().value_counts()

restaurant    29
bar           13
cafe           9
Name: amenity, dtype: int64

In [7]:
len(test[test['amenity']=='restaurant'].dropna(axis=1, how='any'))

29

In [43]:
def pois_amenities(dataset):
    tags = {'amenity':['restaurant', 'bar', 'cafe','university']}
    restaurants = []
    bars = []
    cafes = []
    unis = []
    pois.data = dataset
    for x in range(len(pois.data)):
        pois.temp = ox.pois_from_point(point=(dataset.latitude.iloc[x],dataset.longitude.iloc[x]), dist=1000, tags=tags)  
        if 'amenity' in pois.temp:
            restaurants.append(len(pois.temp[pois.temp['amenity']=='restaurant'].dropna(axis=1, how='any')))  # if len(pois.temp[pois.temp['amenity']=='restaurant'].dropna(axis=1, how='any')) > 0 else restaurants.append(0) 
            bars.append(len(pois.temp[pois.temp['amenity']=='bar'].dropna(axis=1, how='any')))
            cafes.append(len(pois.temp[pois.temp['amenity']=='cafe'].dropna(axis=1, how='any')))
            unis.append(len(pois.temp[pois.temp['amenity']=='university'].dropna(axis=1, how='any')))
        else:
            restaurants.append(0)  # if len(pois.temp[pois.temp['amenity']=='restaurant'].dropna(axis=1, how='any')) > 0 else restaurants.append(0) 
            bars.append(0)
            cafes.append(0)
            unis.append(0)
    pois.data['restaurants'] = restaurants 
    pois.data['bars'] = bars 
    pois.data['cafes'] = cafes
    pois.data['universities'] = unis
    return pois.data
    return pois.temp
    

In [ ]:
def pois_subway(dataset):
    tags = {'railway': 'subway'}
    cta = []
    pois.data = dataset
    for x in range(len(pois.data)):
        pois.temp = ox.pois_from_point(point=(dataset.latitude.iloc[x],dataset.longitude.iloc[x]), dist=1000, tags=tags)  
        if 'subway' in pois.temp:
            train = pois.temp[pois.temp['subway'] == 'yes'].dropna(axis=1, how='any')
            cta.append((len(train['name'].unique())))
        else:
            cta.append(0)
    #print(len(restaurants))
    pois.data['CTA'] = cta
    return pois.data
    return pois.temp
    

In [44]:
sample = data.sample(20)
pois_amenities(sample)

C:\Users\sgevo\Anaconda3\envs\ox\lib\site-packages\osmnx\pois.py:370: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf = gdf.loc[gdf["geometry"].centroid.within(polygon)]
C:\Users\sgevo\Anaconda3\envs\ox\lib\site-packages\osmnx\pois.py:359: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  gdf_ways.crs = settings.default_crs


KeyboardInterrupt: 

In [15]:
pois.temp

,osmid,geometry,highway,traffic_signals,element_type,name,network,operator,public_transport,railway,...,layer,level,oneway,tunnel,voltage,building,building:levels,chicago:building_id,wikidata,wikipedia
736035820,736035820,POINT (-87.66718 41.90356),NaN,NaN,node,Division,CTA,Chicago Transit Authority,stop_position,stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2389087640,2389087640,POINT (-87.64767 41.89114),NaN,NaN,node,Grand,CTA,Chicago Transit Authority,stop_position,stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2389087645,2389087645,POINT (-87.65536 41.89616),NaN,NaN,node,Chicago,CTA,Chicago Transit Authority,stop_position,stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2389087653,2389087653,POINT (-87.66721 41.90352),NaN,NaN,node,Division,CTA,Chicago Transit Authority,stop_position,stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2496841186,2496841186,POINT (-87.66709 41.89634),NaN,NaN,node,Taqueria Traspasada,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209847967,209847967,"POLYGON ((-87.66945 41.90290, -87.66946 41.903...",NaN,NaN,way,Rite Liquors,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,yes,2,322703,NaN,NaN
209853103,209853103,"POLYGON ((-87.66904 41.90453, -87.66916 41.904...",NaN,NaN,way,Tocco,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,yes,1,319502,NaN,NaN
209863979,209863979,"POLYGON ((-87.66741 41.90742, -87.66741 41.907...",NaN,NaN,way,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,yes,2,313398,NaN,NaN
210269961,210269961,"POLYGON ((-87.65995 41.89074, -87.65995 41.890...",NaN,NaN,way,Coalfire,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,yes,NaN,348949,NaN,NaN


In [11]:
import openrouteservice as ors
import folium

In [12]:
client = ors.Client(key='5b3ce3597851110001cf6248fb1ecfcd8ab8422b93b5dd978d83e93e')

In [16]:
trains = pois.temp[pois.temp['subway'] == 'yes'].dropna(axis=1, how='any')
trains

,osmid,geometry,element_type,name,network,operator,public_transport,railway,subway
736035820,736035820,POINT (-87.66718 41.90356),node,Division,CTA,Chicago Transit Authority,stop_position,stop,yes
2389087640,2389087640,POINT (-87.64767 41.89114),node,Grand,CTA,Chicago Transit Authority,stop_position,stop,yes
2389087645,2389087645,POINT (-87.65536 41.89616),node,Chicago,CTA,Chicago Transit Authority,stop_position,stop,yes
2389087653,2389087653,POINT (-87.66721 41.90352),node,Division,CTA,Chicago Transit Authority,stop_position,stop,yes
5272952064,5272952064,POINT (-87.64764 41.89122),node,Grand,CTA,Chicago Transit Authority,stop_position,stop,yes
5272957251,5272957251,POINT (-87.65530 41.89621),node,Chicago,CTA,Chicago Transit Authority,stop_position,stop,yes


In [17]:
loc_str = str(trains.geometry.iloc[0])
import re
loc_num = re.sub("[^0-9,^.,^ ,-]", "", loc_str).lstrip().split(' ')
for i in range(len(loc_num)): 
    loc_num[i] = float(loc_num[i]) 
loc_num

[-87.6671796, 41.9035623]

In [18]:
coordinates = [[sample.iloc[-1].longitude,sample.iloc[-1].latitude]]
coordinates.append(loc_num)

In [19]:
coordinates

[[-87.6574, 41.89965], [-87.6671796, 41.9035623]]

In [20]:
matrix = client.distance_matrix(
    locations=coordinates,
    profile='foot-walking',
    metrics=['distance', 'duration'],
    validate=False,
)

In [21]:
print("Durations in secs: {}\n".format(matrix['durations']))
print("Distances in m: {}".format(matrix['distances']))


Durations in secs: [[0.0, 764.63], [764.63, 0.0]]

Distances in m: [[0.0, 1061.99], [1061.99, 0.0]]


In [22]:
tags_ext = {'amenity':['restaurant', 'bar', 'cafe','university','pub'],
        'railway': 'subway'}
test = ox.pois_from_point(point=(data.loc[33017791].latitude,data.loc[33017791].longitude), dist=1000, tags=tags_ext)  

C:\Users\sgevo\Anaconda3\envs\ox\lib\site-packages\osmnx\pois.py:370: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf = gdf.loc[gdf["geometry"].centroid.within(polygon)]


In [23]:
test[test['amenity'] == 'bar'].dropna(axis=1, how='any')

,osmid,geometry,element_type,name,addr:city,addr:housenumber,addr:postcode,addr:state,addr:street,amenity,phone,website
6265177764,6265177764,POINT (-87.83560 41.98188),node,Bar Louie,Chicago,5615,60631,IL,North Cumberland Avenue,bar,+1-773-332-8029,https://barlouie.com/


In [24]:
data.loc[33017791].latitude

41.97651

In [36]:
coordinates = [[sample.iloc[-1].longitude,sample.iloc[-1].latitude]]
for x in range (len(trains)):
    loc_str = str(trains.geometry.iloc[x])
    loc_num = re.sub("[^0-9,^.,^ ,-]", "", loc_str).lstrip().split(' ')
    for i in range(len(loc_num)): 
        loc_num[i] = float(loc_num[i]) 
    coordinates.append(loc_num)
coordinates

[[-87.6574, 41.89965],
 [-87.6671796, 41.9035623],
 [-87.6476749, 41.8911449],
 [-87.6553624, 41.8961627],
 [-87.6672111, 41.903519],
 [-87.6476367, 41.891221],
 [-87.6553004, 41.8962122]]

In [37]:
coordinates 

[[-87.6574, 41.89965],
 [-87.6671796, 41.9035623],
 [-87.6476749, 41.8911449],
 [-87.6553624, 41.8961627],
 [-87.6672111, 41.903519],
 [-87.6476367, 41.891221],
 [-87.6553004, 41.8962122]]

In [40]:
matrix = client.distance_matrix(
    locations = coordinates,
    destinations = [_ for _ in range(1,len(coordinates))],
    profile = 'foot-walking',
    metrics = ['distance', 'duration'],
    validate = False,
)
print("Durations in secs: {}\n".format(matrix['durations'][0]))
print("Distances in m: {}".format(matrix['distances'][0]))

Durations in secs: [764.63, 1078.5, 368.3, 741.11, 1076.82, 368.57]

Distances in m: [1061.99, 1497.94, 511.54, 1029.33, 1495.61, 511.91]


In [41]:
matrix['durations'][0].index(min(matrix['durations'][0]))

2

In [ ]:
_ = 

In [ ]:
_